In [72]:
#Importing libraries
import numpy as np 
import pandas as pd 
import io
import matplotlib.pyplot as plt

In [73]:
# reading the csv file, del 2 columns from the file, checking first few rows of the file
from google.colab import files
# uploaded = files.upload()

data = pd.read_csv('./BuyComputer.csv')

data.drop(columns=['User ID',],axis=1,inplace=True)
data

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0
...,...,...,...
395,46,41000,1
396,51,23000,1
397,50,20000,1
398,36,33000,0


In [74]:
#Declare label as last column in the source file
from sklearn.model_selection import train_test_split

y=data.iloc[:,-1].values
x=data.iloc[:,:-1].values
n_samples, n_features = x.shape

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [75]:
from sklearn.preprocessing import StandardScaler
import torch

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [76]:
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
print(y_train.shape[0])
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
# print(y_train)
# print(y_test)


320


In [77]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

In [78]:
num_epochs = 140
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [79]:
for epoch in range(num_epochs):
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


with torch.no_grad():
    y_predicted = model(x_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'\n Accuracy: {acc.item()*100:.2f}')

epoch: 10, loss = 0.5832
epoch: 20, loss = 0.5761
epoch: 30, loss = 0.5693
epoch: 40, loss = 0.5629
epoch: 50, loss = 0.5567
epoch: 60, loss = 0.5509
epoch: 70, loss = 0.5453
epoch: 80, loss = 0.5400
epoch: 90, loss = 0.5349
epoch: 100, loss = 0.5301
epoch: 110, loss = 0.5254
epoch: 120, loss = 0.5210
epoch: 130, loss = 0.5168
epoch: 140, loss = 0.5127

 Accuracy: 92.50
